In [1]:
import os 
import torch
from torch import nn
from torchvision import models, transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
from PIL import Image

In [2]:
from data_loader import DatasetLoader
from custom_data import PrivacyDataset
from feature_generation import FeatureGeneration

In [3]:
annotation_path='data/val/val_annotations'
img_path='data/val/val_images'

In [4]:
# Loading data with labels
loader = DatasetLoader()
loaded_data = loader.load_data(annotation_path)[:100]

In [5]:
# Define relevant transformations inc. converting images to tensors
transform = transforms.Compose([transforms.ToPILImage(),
                                transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                #transforms.Normalize((0.5,), (0.5,))
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                ])

In [6]:
dataset=PrivacyDataset(path=img_path,loaded_data=loaded_data,transform=transform)

In [7]:
loader=DataLoader(dataset=dataset,batch_size=len(dataset),shuffle=False)

#### **Testing output of custom dataset**

In [8]:
imgs, labels, img_ids = next(iter(loader))

#### **Apply DenseNet121 pretrained on ImageNet1000**

In [9]:
# Get model
imagenet_model = models.densenet121(pretrained=True)

In [10]:
# Get classes
with open("imagenet_classes.txt", "r") as f:
    imagenet_classes = [s.strip() for s in f.readlines()]

In [11]:
fg = FeatureGeneration(model=imagenet_model,classes=imagenet_classes,img_ids=img_ids)
fg.run_model(imgs)
fg.evaluate(num_results=5)
results = fg.get_results()

In [12]:
# assert(len(results)==40)
assert(loaded_data['id'].iloc[0]==results[0]['id'])

In [13]:
# filter_array = results[0]['probabilities'] > 0.5

In [14]:
# indexes=[filter_array.tolist().index(x) for x in filter_array if x == True]
# classes = list(map(results[0]['class'].__getitem__,indexes))

In [15]:
# indexes, classes

#### **Apply ResNet121 pretrained on Places365**

In [16]:
arch = 'resnet18'                                                                                        
                                                                                                         
model_file = '%s_places365.pth.tar' % arch                                                               
if not os.access(model_file, os.W_OK):                                                                   
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file                           
    os.system('wget ' + weight_url)                                                                      
                                                                                                         
places_model = models.__dict__[arch](num_classes=365)                                                           
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)                           
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}                
places_model.load_state_dict(state_dict)
places_model.eval();

In [17]:
file_name = 'categories_places365.txt'                                                                   
if not os.access(file_name, os.W_OK):                                                                    
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'                                                                                                        
    os.system('wget ' + synset_url)                                                                      
places365_classes = list()                                                                                         
with open(file_name) as class_file:                                                                      
    for line in class_file:                                                                              
        places365_classes.append(line.strip().split(' ')[0][3:])   

In [18]:
fg = FeatureGeneration(model=places_model,classes=places365_classes,img_ids=img_ids)
fg.run_model(imgs)
fg.evaluate(num_results=5)
places_results = fg.get_results()

In [19]:
places_results[:5]

[{'id': '2017_10735550',
  'prob': array([0.5703849 , 0.12074305, 0.03949236, 0.03259051, 0.02637666],
        dtype=float32),
  'class': ['museum/indoor',
   'kindergarden_classroom',
   'kasbah',
   'burial_chamber',
   'archive']},
 {'id': '2017_48376799',
  'prob': array([0.13845207, 0.10528719, 0.06888503, 0.06818575, 0.05153232],
        dtype=float32),
  'class': ['repair_shop',
   'archive',
   'office_cubicles',
   'hardware_store',
   'boat_deck']},
 {'id': '2017_56926780',
  'prob': array([0.2197578 , 0.10150417, 0.08859576, 0.08365171, 0.06590996],
        dtype=float32),
  'class': ['playroom',
   'discotheque',
   'amusement_arcade',
   'recreation_room',
   'stage/indoor']},
 {'id': '2017_67658695',
  'prob': array([0.29997098, 0.29271132, 0.08450764, 0.07293603, 0.05214753],
        dtype=float32),
  'class': ['bedroom',
   'desert/vegetation',
   'rock_arch',
   'hotel_room',
   'bedchamber']},
 {'id': '2017_77918314',
  'prob': array([0.21105438, 0.20612699, 0.0879712